## Calculating Transfer Function between two TE accelerometers with a labJack T7-Pro
Uses 9 analog inputs (AINs) to read the data at 1000 Hz.

Craig Lage - Sep 17, 2021


In [ ]:
import sys
import time, datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.fft import fft, fftfreq, ifft, irfft
from scipy.interpolate import splprep, splrep, splev
from scipy.signal import TransferFunction as tf
from scipy.signal import dlsim, csd, welch
from scipy.optimize import minimize

from labjack import ljm  # Needed pip install labjack-ljm

In [ ]:
scanRate = 1000
readTime = 10.0
df = pd.read_pickle("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6.pkl")

In [ ]:
# Plot the data
plt.figure(figsize=(16,8))
plt.subplots_adjust(hspace=0.5)

for n, scanIndex in enumerate([[0, int(scanRate*readTime-1)], [450, 750], [500,600]]):

    sub_df = df[df.index[scanIndex[0]] : df.index[scanIndex[1]]]
    
    plt.subplot(2,3,n+1)
    ax1 = sub_df['a1z'].plot(label="a1z", color='red')
    ax1.set_title("Axis 1", fontsize=16)
    ax1.set_ylabel("Acceleration(g)")
    ax1.legend(loc='center left')

    plt.subplot(2,3,n+4)
    ax2 = sub_df['a2z'].plot(label="a2z", color='red')
    ax2.set_title("Axis 2", fontsize=16)
    ax2.set_ylabel("Acceleration(g)")
    ax2.legend(loc='center left')
    
    if n == 0: # save this data for the FFT
        a1z = np.array(sub_df['a1z'].tolist())
        a2z = np.array(sub_df['a2z'].tolist())

#plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6_14Sep21.pdf")

In [ ]:
Pyy = csd(a2z, a2z, fs=1000)
Pxy = csd(a1z, a2z, fs=1000)
tf =  Pyy[1] / Pxy[1] # H2 Pyy / Pxy
#tf =  csd(a2z, a1z, fs=1000)[1] / csd(a1z, a1z, fs=1000)[1] # H1 doesn't work??
#tf =  np.conj(crossSpectralDensity[1]) / welchSpectralDensity1[1] # H1 doesn't work??
plt.figure(figsize=(12,3))
plt.subplot(1,4,1)
tck, u = splprep([Pyy[0],np.real(tf)],u=Pyy[0],s=20,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Real(TF)", fontsize=24)
plt.plot(Pyy[0], np.real(tf), marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,2)
tck, u = splprep([Pyy[0],np.imag(tf)],u=Pyy[0],s=20,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Imag(TF)", fontsize=24)
plt.plot(Pyy[0], np.imag(tf), marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,3)
tck, u = splprep([Pyy[0],np.abs(tf)],u=Pyy[0],s=2,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Mag(TF)", fontsize=24)
plt.plot(Pyy[0], np.abs(tf), marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,4)
tck, u = splprep([Pyy[0],np.angle(tf, deg=True)],u=Pyy[0],s=2,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Phase(TF)", fontsize=24)
plt.plot(Pyy[0], np.angle(tf, deg=True), marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)

In [ ]:
G = 0.5 + 0.5 * np.tanh(0.12 * (95.0 - Pyy[0])) + 1j * 3.0 / np.cosh(0.12 * (95.0 - Pyy[0]))
plt.figure(figsize=(12,3))
plt.subplot(1,4,1)
tck, u = splprep([Pyy[0],np.real(tf)],u=Pyy[0],s=20,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Real(TF)", fontsize=24)
plt.plot(Pyy[0], np.real(tf), marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.plot(Pyy[0], np.real(G), color = 'green', label = 'G')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,2)
tck, u = splprep([Pyy[0],np.imag(tf)],u=Pyy[0],s=20,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Imag(TF)", fontsize=24)
plt.plot(Pyy[0], np.imag(tf), marker='x', color='blue', label = 'TF')
plt.plot(Pyy[0], np.imag(G), color = 'green', label = 'G')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,3)
tck, u = splprep([Pyy[0],np.abs(tf)],u=Pyy[0],s=2,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Mag(TF)", fontsize=24)
plt.plot(Pyy[0], np.abs(tf), marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.plot(Pyy[0], np.abs(G), color = 'green', label = 'G')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,4)
tck, u = splprep([Pyy[0],np.angle(tf, deg=True)],u=Pyy[0],s=2,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Phase(TF)", fontsize=24)
plt.plot(Pyy[0], np.angle(tf, deg=True), marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.plot(Pyy[0], np.angle(G, deg=True), color = 'green', label = 'G')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)

In [ ]:
# I don't know how, but this is getting really close!
xp = 19.0 - Pyy[0] / 5.0
G = (1.8 * xp + 3.0 * 1j) / (1.0 + xp * xp / 3.0)
plt.figure(figsize=(12,3))
plt.subplot(1,4,1)
tck, u = splprep([Pyy[0],np.real(tf)],u=Pyy[0],s=20,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Real(TF)", fontsize=24)
plt.plot(Pyy[0], np.real(tf), marker='x', color='blue', label = 'TF')
#plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.plot(Pyy[0], np.real(G), color = 'red', lw = 2.0, label = 'G')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,2)
tck, u = splprep([Pyy[0],np.imag(tf)],u=Pyy[0],s=20,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Imag(TF)", fontsize=24)
plt.plot(Pyy[0], np.imag(tf), marker='x', color='blue', label = 'TF')
plt.plot(Pyy[0], np.imag(G), color = 'red', lw = 2.0, label = 'G')
#plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,3)
tck, u = splprep([Pyy[0],np.abs(tf)],u=Pyy[0],s=2,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Mag(TF)", fontsize=24)
plt.plot(Pyy[0], np.abs(tf), marker='x', color='blue', label = 'TF')
#plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.plot(Pyy[0], np.abs(G), color = 'red', lw = 2.0, label = 'G')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,4,4)
tck, u = splprep([Pyy[0],np.angle(tf, deg=True)],u=Pyy[0],s=2,k=3)
fFit, tfFit = splev(u,tck)
plt.title("Phase(TF)", fontsize=24)
plt.plot(Pyy[0], np.angle(tf, deg=True), marker='x', color='blue', label = 'TF')
#plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.plot(Pyy[0], np.angle(G, deg=True), color = 'red', lw = 2.0, label = 'G')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)

In [ ]:
param0 = [10.0, 0.2, 0.001, 10.0, 0.0001, 0.0003]

def TFFit(params):
    result = 0.0
    [b0, b1, a0, a1] = params
    N = len(Pyy[0])

    def G(z):
        return (b0 + b1 * z + b2 * z * z) / (a0 + a1 * z + a2 * z * z)

    for k in range(N):
        reDiff = np.real(tf[k]) - np.real(G(Pyy[0][k]))
        imDiff = np.imag(tf[k]) - np.imag(G(Pyy[0][k]))
        result += reDiff * reDiff + imDiff * imDiff
    #print(result)
    return result
Result = minimize(TFFit, param0, method='BFGS')
#print(Result)
print(Result.x)
#num = [1.0, Result.x[1], Result.x[0]]
#den = [1.0, Result.x[3], Result.x[2]] 


In [ ]:
G[Pyy[17]]

In [ ]:
def invertTF(tf):
    N = len(tf)
    fullTF = []
    for i in range(N):
        fullTF.append(tf[i])
    for i in range(N-2, 0, -1):
        fullTF.append(np.conj(tf[i]))
    return fullTF
        
    

In [ ]:
test = [0,1,2,3,4]

In [ ]:
test = ifft(invertTF(tf))

In [ ]:
plt.plot(test)

In [ ]:
x = np.linspace(2.0, 200.0, 200)
xp = 9.5 - x / 10.0
plt.subplot(1,2,1)
G1 = 3.0 / np.cosh(xp)
G2 = 3.0 / (1.0 + xp * xp / 2.0)# + xp * xp * xp * xp / 24.0)
plt.plot(x, G1)
plt.plot(x, G2)
plt.subplot(1,2,2)
G1 = 0.5 + 0.5 * np.tanh(xp)
G2 = 0.5 * xp / (1.0 + xp * xp / 3.0)
plt.plot(x, G1)
plt.plot(x, G2)